# Setup & Utils

In [ ]:
import requests

In [ ]:
import re

def extract_model_response(text: str) -> str:
    """
    Extracts the content between <think> and the final <|eot_id|> marker.

    Parameters:
        text (str): The full text containing <think> and <|eot_id|> markers.

    Returns:
        str: The extracted content between <think> and <|eot_id|>, or an empty string if not found.
    """
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", text, re.DOTALL)
    return match.group(1).strip() if match else ""


**Please enter the API key that was provided to you**

In [ ]:
API_KEY = ""

# Use Case - Configuration Assessment

**Use Case Description**- In nearly every security incident, misconfiguration is either the root cause or a critical enabler. Today, most config assessments rely on static rule-matching engines that check config files or cloud settings against predefined rules. But these tools lack contextual understanding of how different components interact — they can't tell if a setting is safe for your environment. This use case focuses on an AI model that ingests configuration data across cloud, infra, app, and identity layers — and reasons over them to answer:

## Config Assessment Utilities

In [ ]:
def create_config_assessment_prompt(config_text):
    """Generate a prompt for security config analysis."""
    prompt = f"""You are a security auditor reviewing the configuration of a Terraform configuration file for security issues.

Go over the following configuration line-by-line and assess it for:
1. Security misconfigurations
2. Weak or deprecated settings
3. Missing best practices

## CONFIGURATION
{config_text}

Respond with:
- Detected misconfiguration
- Severity (Low/Medium/High)
- Recommended fix
"""
    return prompt

In [ ]:
def run_config_assessment(config_text):
     """Send the config to an AI model and return a security assessment."""
     prompt = create_config_assessment_prompt(config_text)
     resp = requests.post(
        "https://model-8w6j1ojq.api.baseten.co/development/predict",
        headers={"Authorization": f"Api-Key {API_KEY}"},
        json={"prompt": prompt},
        )
     return extract_model_response(resp.text)

## Example Usage

Here’s a mock configuration for a Linux-based SSH server:

In [ ]:
terraform_sample = """
provider "google" {
  project = "my-project"
  region  = "us-central1"
}

resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}

resource "google_compute_firewall" "allow_all_inbound" {
  name    = "allow-all-inbound"
  network = google_compute_network.default.name

  direction = "INGRESS"
  priority  = 1000

  allows {
    protocol = "tcp"
    ports    = ["0-65535"]
  }

  allows {
    protocol = "udp"
    ports    = ["0-65535"]
  }

  source_ranges = ["0.0.0.0/0"]

  target_tags = ["web"]
  description = "Allow all inbound traffic from any IP on all ports"
}
"""


In [ ]:
output = run_config_assessment(terraform_sample)

In [ ]:
print(output)

I'll review the provided Terraform configuration line-by-line, identifying potential security misconfigurations, weak or deprecated settings, and missing best practices.

### Provider Configuration

```haskell
provider "google" {
  project = "my-project"
  region  = "us-central1"
}
```

* No immediate security concerns here. The provider is correctly configured with a project and region.
* Best practice: Consider using a service account key file instead of hardcoding credentials. However, since this isn't mentioned in the code snippet, I won't mark it as a detected issue here. But it's worth noting in a real-world audit.

### Network Resource

```haskel
resource "google_compute_network" "default" {
  name                    = "default-network"
  auto_create_subnetworks = true
}
```

* Misconfiguration: **Medium** - Allowing auto-create subnets might lead to unnecessary resources being created. It's recommended to set `auto_create_subnetworks` to `false` unless explicitly needed. This c